<a href="https://colab.research.google.com/github/comar80/tradutor-azure/blob/main/Tradutor_por_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip  install requests beautifulsoup4 openai langchain-openai

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
  response = requests.get(url)

  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')

      for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()

      text = soup.get_text(separator=' ')

      lines = (line.strip() for line in text.splitlines())
      parts = (phrase.strip() for line in lines for phrase in line.split(" "))
      text_clean = '\n'.join(part for part in parts if part)
      return text_clean

  else:
    print(f"Falhou. Status: {response.status_code}")
    return None

  text = soup.get_text()
  return text

extract_text_from_url('https://dev.to/nfrankel/duckdb-in-action-4d29')


In [15]:
from langchain_openai.chat_models.azure import AzureChatOpenAI
from google.colab import userdata
api_key = userdata.get('api_key')



client = AzureChatOpenAI(
    azure_endpoint = 'https://openai-dio-dev2.openai.azure.com/',
    api_key = api_key,
    api_version = '2024-02-15-preview',
    deployment_name = 'gpt-4o-mini',
    max_retries=0
)

def translate_article(text, lang):
  messages = [
      ("system", "Você é um tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
  ]

  response = client.invoke(messages)
  print(response.content)
  return response.content

translate_article("The new $599 M4 Mac Mini is really good. In more than one way.", "portugues")

O novo Mac Mini M4 de $599 é realmente bom. De mais de uma maneira.


'O novo Mac Mini M4 de $599 é realmente bom. De mais de uma maneira.'

In [ ]:
url = "https://dev.to/nfrankel/duckdb-in-action-4d29"
text = extract_text_from_url(url)
article = translate_article(text, "pt-br")

print(article)